In [6]:
import numpy as np 
import pandas as pd
import pickle
from sklearn import datasets

In [7]:
# import river modules
from river import evaluate
from river import metrics
from river import optim
from river import stream
from river import drift

#### Drift Detection using Pagehinkley

In [8]:
# Initializing drift detector
ph = drift.PageHinkley()
ph

PageHinkley (
  min_instances=30
  delta=0.005
  threshold=50.
  alpha=0.9999
  mode="both"
)

In [9]:
# Creating regression dataset from sklearn dataset module
np.random.seed(1000)
X, y = datasets.make_regression(n_samples=100, n_features=4)

# Loading model from pickle file
PARegmodel = pickle.load(open('saved_models/regression/PARegmodel.pkl', 'rb'))
metric = metrics.MAE() + metrics.MSE()

In [10]:
for xi, yi in stream.iter_array(X, y):
    # Data is processed one sample at a time
    y_pred = PARegmodel.predict_one(xi)
    metric = metric.update(yi, y_pred)
    error = yi-y_pred
    # Update drift detector with error
    ph.update(error)
    if ph.drift_detected:
         print(f"Change detected with data: {xi}")
print("\n")
print(metric)

Change detected with data: {0: 1.0485307227685718, 1: -0.8353282327925169, 2: 1.1334172311772497, 3: -1.694998315517214}
Change detected with data: {0: 1.3944227462859724, 1: -2.12832537042671, 2: 0.1779371057504889, 3: -1.4371393945822026}
Change detected with data: {0: -0.3831474108842978, 1: 0.3121180100663634, 2: -0.5501738650283003, 3: 1.5331610653579348}


MAE: 89.620452, MSE: 12,602.256485
